[git laba3](https://github.com/newprolab/sber-spark-ds-10/blob/main/labs/lab03.md)

### Лаба 3. Рекомендательная система видеоконтента с implicit feedback – Spark ML

В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами покупок абонентами телепередач от компании E-Contenta. По доступным вам данным, нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами. При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и другие. Если scikit-learn (например, но и другие тоже) обернут в классы Transformer и Estimator, то их можно использовать.

**Результат**

Предсказание целевой переменной "купит/не купит" — хорошо знакомая вам задача бинарной классификации. Поскольку нам важны именно вероятности отнесения пары (пользователь, товар) к классу "купит" (`1`), то, на самом деле, вы можете подойти к проблеме с разных сторон:

1. Как просто к задаче бинарной классификации. У вас есть два датасета, которые можно каким-то образом объединить, дополнительно обработать и сделать предсказания классификаторами (Spark ML). 
2. Как к разработке рекомендательной системы: рекомендовать пользователю `user_id` топ-N лучших телепередач, которые были найдены по методике user-user / item-item коллаборативной фильтрации. 
3. Как к задаче факторизации матриц: алгоритмы SVD, ALS, FM/FFM.


**Советы**

1. На качество прогноза в большей степени влияет качество признаков, которые вы сможете придумать из имеющихся данных, нежели выбор и сложность алгоритма.
2. Качество входных данных также имеет сильное значение. Существует фраза "garbage in – garbage out". Мусор на входе – мусор на выходе. Потратьте время на подготовку и предобработку данных. Путь к успеху в третьей лабораторной:
3. Сосредоточьтесь на формировании следующих фичей: по файлу laba03_train.csv сформируйте признаки, характеризирующие как интенсивно покупает пользователь и "покупаемость" item'ов
4. возьмите достаточно мощную модель (например GBTClassifier из pyspark'а)

In [6]:
import os
import sys
import re
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [7]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, ArrayType, BinaryType
from pyspark.sql.types import ByteType
from pyspark.ml.feature import CountVectorizer

import json
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.linalg import Vectors
import pyspark.sql.functions as F
from pyspark.sql.window import Window

from pyspark.ml.feature import  VectorAssembler
from pyspark.ml import Pipeline

from pyspark.ml.classification import GBTClassifier

conf = SparkConf()
conf.set("spark.app.name", "Kurseev Maxim Spark 3 lab") 

spark = SparkSession.builder.config(conf=conf).appName("Kurseev Maxim Spark 3 lab").getOrCreate()

In [8]:
spark

**Train and Test files**

In [9]:
schema_train = StructType(fields=[
    StructField('user_id', IntegerType()),
    StructField('item_id', IntegerType()),
    StructField('purchase', ByteType())
])

schema_test = StructType(fields=[
    StructField('user_id', IntegerType()),
    StructField('item_id', IntegerType()),
])

train = spark.read.csv('/labs/slaba03/laba03_train.csv', header=True, schema=schema_train)
test = spark.read.csv('/labs/slaba03/laba03_test.csv', header=True, schema=schema_test)

In [10]:
test.show(3)
train.show(3)

+-------+-------+
|user_id|item_id|
+-------+-------+
|   1654|  94814|
|   1654|  93629|
|   1654|   9980|
+-------+-------+
only showing top 3 rows

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
+-------+-------+--------+
only showing top 3 rows



**items**

`item_id` — primary key. Соответствует item_id в предыдущем файле.

`content_type` — тип телепередачи (1 — платная, 0 — бесплатная). Вас интересуют платные передачи.

`title` — название передачи, текстовое поле.

`year` — год выпуска передачи, число.

`genres` — поле с жанрами передачи, разделёнными через запятую.

In [11]:
schema_item = StructType(fields=[
    StructField('item_id', IntegerType()),
    StructField('channel_id', StringType()),
    StructField('datetime_availability_start', StringType()),
    StructField('datetime_availability_stop', StringType()),
    StructField('datetime_show_start', StringType()),
    StructField('datetime_show_stop', StringType()),
    StructField('content_type', IntegerType()),
    StructField('title', StringType()),
    StructField('year', IntegerType()),
    StructField('genres', StringType()),
    StructField('region_id', IntegerType()),
])

items = spark.read.csv('/labs/slaba03/laba03_items.csv', header=True, sep='\t',schema=schema_item)
items = (items
         .filter(
             (~F.col('item_id').isNull())
             & (F.col('content_type') == 1)
         )
         .drop('channel_id','region_id','datetime_show_start','datetime_availability_start',
               'datetime_availability_stop','datetime_show_stop', 'content_type', 'title')
         .na.fill(value=2010,subset=["year"])
         .na.fill(value='Мультфильмы',subset=["genres"])
         .drop('year') ## потому что только 2010 остается
        )

items.show(1,False,True)

-RECORD 0----------
 item_id | 65667   
 genres  | Эротика 
only showing top 1 row



In [12]:
items.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in items.columns]).show()

+-------+------+
|item_id|genres|
+-------+------+
|      0|     0|
+-------+------+



**views_programmes**

In [13]:
## нет в items активных ни одного совпадения

read_users_schema = StructType(fields=[StructField('user_id', IntegerType()), 
StructField('item_id', IntegerType()),
StructField('ts_start', IntegerType()),
StructField('ts_end', IntegerType()),
StructField('item_type', StringType()),
]) 



views_programmes = spark.read.format("csv") \
      .option("header", True) \
      .schema(read_users_schema) \
      .load("/labs/slaba03/laba03_views_programmes.csv")

In [14]:
views_programmes.show(1,False,True)

-RECORD 0---------------
 user_id   | 0          
 item_id   | 7101053    
 ts_start  | 1491409931 
 ts_end    | 1491411600 
 item_type | live       
only showing top 1 row



In [15]:
## func 
genres_split = F.udf(lambda x: x.split(','), ArrayType(StringType()))
items = items.withColumn('genres', genres_split(F.col('genres')))

cv = CountVectorizer(inputCol="genres", outputCol="genres_features")
model = cv.fit(items)
items = model.transform(items).drop('genres')
items.show(5, truncate=False)

+-------+---------------+
|item_id|genres_features|
+-------+---------------+
|65667  |(83,[20],[1.0])|
|65669  |(83,[20],[1.0])|
|65668  |(83,[20],[1.0])|
|65671  |(83,[20],[1.0])|
|65670  |(83,[20],[1.0])|
+-------+---------------+
only showing top 5 rows



In [16]:
windowSpecAgg  = Window.partitionBy('user_id')
train = train.withColumn('user_activity', F.sum('purchase').over(windowSpecAgg)/F.count('purchase').over(windowSpecAgg))

windowSpecAgg  = Window.partitionBy('item_id')
train = train.withColumn('items_intenstiy', F.sum('purchase').over(windowSpecAgg)/F.count('purchase').over(windowSpecAgg))

train = train.join(items, how='left', on='item_id')

# check for nan colums
cols=["item_id","user_id", "purchase", "user_activity", "items_intenstiy"]
train.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in cols]).show()

In [19]:
train.show(5)

+-------+-------+--------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|       user_activity|     items_intenstiy|     genres_features|
+-------+-------+--------+--------------------+--------------------+--------------------+
|   8389| 754230|       0|0.027575641516660282|0.005979073243647235|(83,[6,13,19,22],...|
|   8389| 780033|       0|7.757951900698216E-4|0.005979073243647235|(83,[6,13,19,22],...|
|   8389| 798454|       0|3.840245775729646...|0.005979073243647235|(83,[6,13,19,22],...|
|   8389| 825061|       0|0.001931247585940...|0.005979073243647235|(83,[6,13,19,22],...|
|   8389| 833685|       0|0.007500986971969996|0.005979073243647235|(83,[6,13,19,22],...|
+-------+-------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



**fitting**

In [20]:
assembler = (
    VectorAssembler()
    .setInputCols(['user_activity', 'items_intenstiy', 'genres_features']) 
    .setOutputCol('features')
)

train_fitting = assembler.transform(train)
train_df, test_df = train_fitting.randomSplit([0.8, 0.2], seed = 123)

In [21]:
train_df.show(3)

+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|purchase|       user_activity|     items_intenstiy|     genres_features|            features|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
|   8389| 520446|       0|0.003053435114503...|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
|   8389| 619378|       0|3.859513701273639...|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
|   8389| 625678|       0|0.005886970172684459|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
+-------+-------+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



* понять через grid_search 2 гиперпараметра `maxIter` и `stepSize`!
* понять что predict работает

In [22]:
maxDepth = 5   # This indicates how deep the built tree can be
maxBins  = 32
maxIter  = 70  # number of trees
stepSize = 0.1 # learing rate

gbt = GBTClassifier(labelCol="purchase", 
                    featuresCol="features",
                    maxDepth=maxDepth,
                    maxBins=maxBins,
                    maxIter=maxIter,
                    stepSize=stepSize,
                   )

model = gbt.fit(train_df)
predictions = model.transform(test_df)

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol = 'purchase', metricName='areaUnderROC')

In [25]:
print('ROC AUC Train', evaluator.evaluate(predictions))

ROC AUC Train 0.9368114083960376


In [37]:
predictions.select('item_id','user_id','purchase','probability').show(5)

+-------+-------+--------+--------------------+
|item_id|user_id|purchase|         probability|
+-------+-------+--------+--------------------+
|   8389| 556825|       0|[0.97772420621832...|
|   8389| 566701|       0|[0.97562889312552...|
|   8389| 613775|       0|[0.98341494594472...|
|   8389| 639612|       0|[0.98209654982554...|
|   8389| 703514|       0|[0.98345630435580...|
+-------+-------+--------+--------------------+
only showing top 5 rows



## predictions

In [40]:
user_activity_df = train.groupBy('user_id').agg(
    (F.sum('purchase')/F.count('purchase')
    ).alias('user_activity'))

item_intensivity_df = train.groupBy('item_id').agg(
    (F.sum('purchase')/F.count('purchase')
    ).alias('item_intensivity'))


test = test.join(user_activity_df, how='left', on='user_id')
test = test.join(item_intensivity_df, how='left', on='item_id')
test = test.join(items, how='left', on='item_id')

In [49]:
assembler_test = (
    VectorAssembler()
    .setInputCols(['user_activity', 'item_intensivity', 'genres_features']) 
    .setOutputCol('features')
)

test_model = assembler_test.transform(test)

In [50]:
test_model.show(4)

+-------+-------+--------------------+--------------------+--------------------+--------------------+
|item_id|user_id|       user_activity|    item_intensivity|     genres_features|            features|
+-------+-------+--------------------+--------------------+--------------------+--------------------+
|   8389| 566758|3.766478342749529E-4|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
|   8389| 816426|                 0.0|0.005979073243647235|(83,[6,13,19,22],...|(85,[1,8,15,21,24...|
|   8389| 892290|7.719027402547279E-4|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
|   8389| 901323|3.846153846153846E-4|0.005979073243647235|(83,[6,13,19,22],...|(85,[0,1,8,15,21,...|
+-------+-------+--------------------+--------------------+--------------------+--------------------+
only showing top 4 rows



In [89]:
predictions_test = model.transform(test_model)

In [90]:
get_prob = F.udf(lambda x: float(x[1]), FloatType())

predictions_test = (predictions_test
                    .withColumn('purchase', get_prob(F.col('probability')))
                    .select('user_id', 'item_id', 'purchase')
                    .orderBy('user_id', 'item_id')
                   )

In [91]:
predictions_test.show(4, False)

+-------+-------+-----------+
|user_id|item_id|purchase   |
+-------+-------+-----------+
|1654   |336    |0.016343297|
|1654   |678    |0.016343297|
|1654   |691    |0.016343297|
|1654   |696    |0.016566597|
+-------+-------+-----------+
only showing top 4 rows



In [92]:
predictions_pandas = predictions_test.toPandas()

In [96]:
predictions_pandas.to_csv('lab03.csv')

___
**Grid search**

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
pipeline = Pipeline(stages=[gbt])

paramGrid = ParamGridBuilder().addGrid(gbt.maxIter, [30, 60])\
                              .addGrid(gbt.stepSize, [0.1, 0.3])\
                              .build()

crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid,
                              evaluator=evaluator, numFolds=4, parallelism=4)

cv_model = crossval.fit(train_fitting)

In [ ]:
cv_model.avgMetrics

In [ ]:
cv_model.bestModel

___
### Stop session

In [97]:
spark.stop()